In [2]:
!pip install imbalanced-learn

In [1]:
import pandas as pd
import numpy as np
from phik.report import plot_correlation_matrix
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, \
    classification_report, roc_auc_score
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [2]:
df_raw = pd.read_csv('/kaggle/input/leopard-challenge-classification/train.csv')
test = pd.read_csv(r'/kaggle/input/leopard-challenge-classification/test.csv')
df_raw.shape, test.shape

((13863, 26), (5942, 25))

In [3]:
test_cv = test.drop(['oral', 'ID'], axis = 1).copy()
test_cv['tartar'] = test_cv['tartar'].apply(lambda x: 1 if x == 'Y' else 0)

In [4]:
merged_data = df_raw.drop(['oral', 'ID'], axis=1).copy()
merged_data['tartar'] = merged_data['tartar'].apply(lambda x: 1 if x == 'Y' else 0)

In [5]:
# Создание бинарного признака на основе BMI
merged_data['BMI'] = merged_data['weight(kg)'] / ((merged_data['height(cm)'] / 100) ** 2)
merged_data['BMI_status'] = (merged_data['BMI'] < 18.5) | (merged_data['BMI'] >= 24.9)
merged_data['BMI_status'] = merged_data['BMI_status'].astype(int)
merged_data['log_tr'] = merged_data['triglyceride'].apply(np.log)
merged_data['log_gtp'] = merged_data['Gtp'].apply(np.log)
merged_data['log_log_alt'] = np.log(np.log(merged_data['ALT']))
merged_data['BMI_log_gtp'] = merged_data['BMI'].apply(np.log)
merged_data = merged_data.drop(['triglyceride', 'Gtp', 'ALT', 'BMI'], axis=1)

test_cv['BMI'] = test_cv['weight(kg)'] / ((test_cv['height(cm)'] / 100) ** 2)
test_cv['BMI_status'] = (test_cv['BMI'] < 18.5) | (test_cv['BMI'] >= 24.9)
test_cv['BMI_status'] = test_cv['BMI_status'].astype(int)
test_cv['log_tr'] = test_cv['triglyceride'].apply(np.log)
test_cv['log_gtp'] = test_cv['Gtp'].apply(np.log)
test_cv['log_log_alt'] = np.log(np.log(test_cv['ALT']))
test_cv['BMI_log_gtp'] = test_cv['BMI'].apply(np.log)
test_cv = test_cv.drop(['triglyceride', 'Gtp', 'ALT', 'BMI'], axis=1)

In [6]:
from sklearn.model_selection import train_test_split
# polynom = PolynomialFeatures(interaction_only=True, include_bias=False)
# from imblearn.over_sampling import SMOTE

X = merged_data.drop(['smoking'], axis=1) 
Y = merged_data['smoking']

# X_poly = pd.DataFrame(polynom.fit_transform(X))
# test_cv_poly = pd.DataFrame(polynom.transform(test_cv))

# Применение SMOTE
# X_new_train, X_test_sim, Y_new_train, Y_test_sim = train_test_split(X, Y, test_size=0.3, random_state=42, stratify=Y)
# X_new_train.shape, X_test_sim.shape, Y_new_train.shape, Y_test_sim.shape

In [7]:
from sklearn.preprocessing import MinMaxScaler

# Масштабирование признаков
scaler = MinMaxScaler()

X_scale = scaler.fit_transform(X)
test_cv = scaler.transform(test_cv)

X = pd.DataFrame(X_scale)
X.shape, test_cv.shape

((13863, 25), (5942, 25))

In [8]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.linear_model import LogisticRegression
from collections import Counter
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.manifold import TSNE
import optuna

In [10]:
import json
import pickle

# Загрузка сохраненных параметров
with open('/kaggle/input/lgbm-optuna-v-1-best-params-no-exta-features/LGBM_Optuna_V_1_best_params_no_exta_features.json', 'r') as f:
    LGBM_best_params = json.load(f)

# Обучение модели с загруженными параметрами
# model_with_best_params = LGBMClassifier(**loaded_params)

# Загрузка гиперпараметров из файла
with open("/kaggle/input/best-xgb-params/best_xgb_params.pkl", "rb") as f:
    loaded_best_xgb_params = pickle.load(f)
    
with open("/kaggle/input/cb-best-parameters/best_parameters.json", "r") as f:
    CatBoost_loaded_best_params = json.load(f)


In [11]:
%%time
ens_cv_scores, ens_preds = list(), list()
hill_ens_cv_scores, hill_ens_preds =  list(), list()

results_df = pd.DataFrame(columns=['Fold', 'Model', 'ROC AUC', 'Optimal Threshold', 'F1 Score'])

ens_cv_scores = []
ens_preds = []

sk = RepeatedStratifiedKFold(n_splits = 7, n_repeats = 1, random_state = 42)
ratio = float(np.sum(Y == 0)) / np.sum(Y == 1)

for i, (train_idx, test_idx) in enumerate(sk.split(X, Y)):

    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
    
    X_train_sub, X_val_sub, Y_train_sub, Y_val_sub = train_test_split(X_train, Y_train, test_size=0.2, random_state=42, stratify=Y_train)
    
    print('----------------------------------------------------------')

    ##########
    ## LGBM ##
    ##########

    LGBM_md = LGBMClassifier(**LGBM_best_params
#                              class_weight='balanced',
#                              n_estimators = 3500,
#                              max_depth = 9,
#                              learning_rate = 0.03,
#                              num_leaves = 20,
#                              reg_alpha = 2,
#                              reg_lambda = 5,
#                              subsample = 0.7,
#                              colsample_bytree = 0.7
                                                    ).fit(X_train_sub, Y_train_sub, 
                                                            eval_set=[(X_val_sub, Y_val_sub)], 
                                                            early_stopping_rounds=100, 
                                                            verbose=100)

    lgb_pred = LGBM_md.predict_proba(X_test)[:, 1]
    lgb_score = roc_auc_score(Y_test, lgb_pred)

    print('Fold', i, '==> LGBM oof ROC-AUC score is ==>', lgb_score) 

    lgb_pred_test = LGBM_md.predict_proba(test_cv)[:, 1]

    #########
    ## XGB ##
    #########

    XGB_md = XGBClassifier(**loaded_best_xgb_params
#                             objective = 'binary:logistic',
#                            tree_method = 'hist',
#                            colsample_bytree = 0.7, 
#                            gamma = 2, 
#                            learning_rate = 0.03, 
#                            max_depth = 10, 
#                            min_child_weight = 10, 
#                            n_estimators = 2500, 
#                            reg_lambda=5,
#                            scale_pos_weight=ratio,
#                            subsample = 0.7
                                  ).fit(X_train_sub, Y_train_sub, 
                                                   eval_metric="auc", 
                                                   eval_set=[(X_val_sub, Y_val_sub)], 
                                                   early_stopping_rounds=100, 
                                                   verbose=100)

    xgb_pred = XGB_md.predict_proba(X_test)[:, 1]
    xgb_score = roc_auc_score(Y_test, xgb_pred)

    print('Fold', i, '==> XGB oof ROC-AUC score is ==>', xgb_score)

    xgb_pred_test = XGB_md.predict_proba(test_cv)[:, 1]

    ##############
    ## CatBoost ##
    ##############

    Cat_md = CatBoostClassifier(**CatBoost_loaded_best_params
#                                 auto_class_weights='Balanced',
#                                 eval_metric='AUC',
#                                 iterations = 3_000,
#                                 learning_rate = 0.05,
#                                 depth = 8,
#                                 random_strength = 0.5,
#                                 bagging_temperature = 0.7,
#                                 border_count = 30,
#                                 l2_leaf_reg = 7,
#                                 verbose = False, 
#                                 task_type = 'CPU'
                                                ).fit(X_train_sub, Y_train_sub, 
                                                           eval_set=(X_val_sub, Y_val_sub), 
                                                           early_stopping_rounds=100, 
                                                           verbose=100, 
                                                           use_best_model=True)

    cat_pred = Cat_md.predict_proba(X_test)[:, 1]
    cat_score = roc_auc_score(Y_test, cat_pred)

    print('Fold', i, '==> CatBoost oof ROC-AUC score is ==>', cat_score)

    cat_pred_test = Cat_md.predict_proba(test_cv)[:, 1]    
    
    ##############
    ## Ensemble ##
    ##############
    
    
    
    
    # Ансамбль моделей
    ens_pred_1 = ( lgb_pred + xgb_pred + cat_pred ) / 3
    ens_pred_2 = ( lgb_pred_test + xgb_pred_test + cat_pred_test ) / 3
    ens_score_fold = roc_auc_score(Y_test, ens_pred_1)
    ens_cv_scores.append(ens_score_fold)
    ens_preds.append(ens_pred_2)

    print('Fold', i, '==> Average Ensemble oof ROC-AUC score is ==>', ens_score_fold)
    
    # Поиск оптимального порога для F1
    thresholds = np.linspace(0, 1, 300)
    f1_scores = [f1_score(Y_test, ens_pred_1 > thresh) for thresh in thresholds]
    optimal_threshold = thresholds[np.argmax(f1_scores)]
    ens_f1_score = max(f1_scores)
    
    results_df.loc[len(results_df)] = [i, 'LGBM', lgb_score, optimal_threshold, ens_f1_score]
    results_df.loc[len(results_df)] = [i, 'XGB', xgb_score, optimal_threshold, ens_f1_score]
    results_df.loc[len(results_df)] = [i, 'CatBoost', cat_score, optimal_threshold, ens_f1_score]
    

ens_mean_score = np.mean(ens_cv_scores)
optimal_threshold, ens_mean_score

----------------------------------------------------------
[LightGBM] [Warning] lambda_l1 is set=3.8476260070128565, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.8476260070128565
[LightGBM] [Warning] bagging_fraction is set=0.4265533399132706, subsample=1.0 will be ignored. Current value: bagging_fraction=0.4265533399132706
[LightGBM] [Warning] lambda_l2 is set=2.465060612958123, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.465060612958123
[LightGBM] [Warning] feature_fraction is set=0.6359736909429626, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6359736909429626
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2


/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's binary_logloss: 0.468304
[200]	valid_0's binary_logloss: 0.459608
[300]	valid_0's binary_logloss: 0.457919
[400]	valid_0's binary_logloss: 0.457052
[500]	valid_0's binary_logloss: 0.455739
Fold 0 ==> LGBM oof ROC-AUC score is ==> 0.7093959519291587
[0]	validation_0-auc:0.62506


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[100]	validation_0-auc:0.71015
[162]	validation_0-auc:0.70908
Fold 0 ==> XGB oof ROC-AUC score is ==> 0.7157827324478179
0:	learn: 0.6868449	test: 0.6869006	best: 0.6869006 (0)	total: 115ms	remaining: 2m 17s
100:	learn: 0.4476326	test: 0.4857462	best: 0.4857462 (100)	total: 4.1s	remaining: 44.4s
200:	learn: 0.3764441	test: 0.4636184	best: 0.4636184 (200)	total: 8.14s	remaining: 40.3s
300:	learn: 0.3294015	test: 0.4567743	best: 0.4567455 (299)	total: 12.2s	remaining: 36.2s
400:	learn: 0.2955431	test: 0.4537018	best: 0.4537018 (400)	total: 16.3s	remaining: 32.3s
500:	learn: 0.2669852	test: 0.4521899	best: 0.4521793 (493)	total: 20.3s	remaining: 28.2s
600:	learn: 0.2406800	test: 0.4511089	best: 0.4511089 (600)	total: 24.4s	remaining: 24.1s
700:	learn: 0.2168064	test: 0.4503129	best: 0.4502209 (690)	total: 28.7s	remaining: 20.3s
800:	learn: 0.1968009	test: 0.4501061	best: 0.4500347 (791)	total: 32.6s	remaining: 16.1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.4500

...

/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[100]	valid_0's binary_logloss: 0.469825
[200]	valid_0's binary_logloss: 0.462244
[300]	valid_0's binary_logloss: 0.459729
[400]	valid_0's binary_logloss: 0.460093
Fold 6 ==> LGBM oof ROC-AUC score is ==> 0.7350316455696202
[0]	validation_0-auc:0.58728


/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[100]	validation_0-auc:0.70858
[200]	validation_0-auc:0.70956
[204]	validation_0-auc:0.70899
Fold 6 ==> XGB oof ROC-AUC score is ==> 0.7255712025316456
0:	learn: 0.6869543	test: 0.6871118	best: 0.6871118 (0)	total: 39.4ms	remaining: 47.1s
100:	learn: 0.4508187	test: 0.4883369	best: 0.4883369 (100)	total: 4.35s	remaining: 47.1s
200:	learn: 0.3789511	test: 0.4663792	best: 0.4663792 (200)	total: 8.38s	remaining: 41.4s
300:	learn: 0.3304091	test: 0.4592494	best: 0.4592494 (300)	total: 12.5s	remaining: 37.1s
400:	learn: 0.2942722	test: 0.4547976	best: 0.4547976 (400)	total: 16.5s	remaining: 32.7s
500:	learn: 0.2647090	test: 0.4524843	best: 0.4524357 (496)	total: 20.6s	remaining: 28.5s
600:	learn: 0.2388793	test: 0.4507755	best: 0.4507755 (600)	total: 24.7s	remaining: 24.4s
700:	learn: 0.2169251	test: 0.4496923	best: 0.4495754 (696)	total: 28.7s	remaining: 20.2s
800:	learn: 0.1954816	test: 0.4493669	best: 0.4493577 (798)	total: 33s	remaining: 16.2s
900:	learn: 0.1771844	test: 0.4500496	best:

(0.1906354515050167, 0.7288154115668318)

In [18]:
results_df

,Fold,Model,ROC AUC,Optimal Threshold,F1 Score
0,0,LGBM,0.703652,0.294314,0.434515
1,0,XGB,0.698606,0.294314,0.434515
2,0,CatBoost,0.702623,0.294314,0.434515
3,1,LGBM,0.676868,0.284281,0.411834
4,1,XGB,0.673497,0.284281,0.411834
5,1,CatBoost,0.681975,0.284281,0.411834
6,2,LGBM,0.716152,0.267559,0.454194
7,2,XGB,0.700759,0.267559,0.454194
8,2,CatBoost,0.715045,0.267559,0.454194
9,3,LGBM,0.717722,0.244147,0.465753


In [12]:
results_df # this is new

,Fold,Model,ROC AUC,Optimal Threshold,F1 Score
0,0,LGBM,0.709396,0.187291,0.452962
1,0,XGB,0.715783,0.187291,0.452962
2,0,CatBoost,0.730982,0.187291,0.452962
3,1,LGBM,0.700441,0.173913,0.440183
4,1,XGB,0.697266,0.173913,0.440183
5,1,CatBoost,0.716244,0.173913,0.440183
6,2,LGBM,0.705663,0.160535,0.446673
7,2,XGB,0.721581,0.160535,0.446673
8,2,CatBoost,0.721795,0.160535,0.446673
9,3,LGBM,0.714818,0.173913,0.456863


In [13]:
avg_ens_preds = np.mean(ens_preds, axis=0)
final_predictions = (avg_ens_preds > optimal_threshold).astype(int)
final_predictions.sum()

1810

In [14]:
final_predictions.shape

(5942,)

In [80]:
from sklearn.metrics import confusion_matrix

# Теперь вы можете сравнить прогнозы с реальными значениями
cm = confusion_matrix(Y_test_sim, final_predictions)

cm

ValueError: Found input variables with inconsistent numbers of samples: [4159, 5942]

In [73]:
# 1. Прогнозирование с помощью каждой отдельной модели на новых данных
lgb_new_pred = LGBM_md.predict_proba(X)[:, 1]
xgb_new_pred = XGB_md.predict_proba(X)[:, 1]
cat_new_pred = Cat_md.predict_proba(X)[:, 1]

# 2. Усреднение прогнозов от разных моделей
ens_new_pred = (lgb_new_pred + xgb_new_pred + cat_new_pred) / 3

# Если вам нужны бинарные прогнозы (0 или 1), вы можете применить порог, который был найден ранее
ens_new_binary_pred = (ens_new_pred > optimal_threshold).astype(int)
ens_new_binary_pred.shape

(13863,)

In [15]:
tester_df = pd.DataFrame()
tester_df['smoking'] = Y
tester_df['smoking_pred'] = ens_new_binary_pred

NameError: name 'ens_new_binary_pred' is not defined

In [16]:
tester_df_cm = confusion_matrix(tester_df['smoking'], tester_df['smoking_pred'])
tester_df_cm

KeyError: 'smoking_pred'

In [17]:
submission = pd.read_csv(r'/kaggle/input/leopard-challenge-classification/sample_submission.csv')

In [20]:
submission['smoking'] = final_predictions
submission.to_csv('Ensemble_submission_08.csv', index = False)

In [19]:
submission['smoking'].sum()

1810